In [1]:
import sys
import pandas as pd
import numpy as np
from typing import Any
from scipy.stats import linregress, ttest_1samp

%load_ext autoreload
%autoreload 2
# sys.path.append("D:/Github/note/module")                        # for windows
sys.path.append("/Users/xinc./Documents/GitHub/note/module")    # for mac
from get_info_FinMind import GetInfoFinMind
FinMind_fetcher = GetInfoFinMind()
from plot_func import plot, plot_scatter, plot_df_columns, plot_pdf, plot_dropped_positions, plot_sequence
from performance_func import summarize_performance, mean_ttest

start_date = pd.to_datetime("2020-01-01", format = "%Y-%m-%d")
end_date = pd.to_datetime("2025-09-25", format = "%Y-%m-%d")